# Solar Data Tutorial Worksheet 
Hello Solar REU interns at the CfA. This is a quick data tutorial, which hopefully will help with your summer reasearch.

## Helioviewer Example
Helioviewer is a useful tool for exploration. Using JHelioviewer in particular may help you find interesting events. Here I will walk you through a quick example with JHelioviewer. 

You can launch JHelioviewer by clicking the JHelioviewer icon (below) in either Applications or your Dock.
<img src="JHV_icon_small.png" />


## Data Download 
There are two primary ways to download data for Solar mission archives: websites and programming scripts.



